<a href="https://colab.research.google.com/github/jasondupree/jasondupree.github.io/blob/main/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA CREATION

In [115]:
import numpy as np
import pandas as pd
import random
import string
from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)

# ✅ Global Parameters for Full Control
NUM_WEEKS = 52
NUM_CHANNELS = 3
NUM_SOLD_TO_NUMBERS = 6
NUM_PRODUCTS = 10
NUM_SIZES = 3

# ✅ Channels - Configurable
CHANNELS = ["NIKE DIRECT DIGITAL", "NIKE DIRECT STORES", "WHOLESALE"][:NUM_CHANNELS]

# ✅ Channel Selling Prices
SELLING_PRICE_BY_CHANNEL = {
    "NIKE DIRECT DIGITAL": 50,
    "NIKE DIRECT STORES": 45,
    "WHOLESALE": 40
}

# ✅ Liquidation Selling Price
LIQUIDATION_SELLING_PRICE = 20

# ✅ End Product Offer Date (EPOD)
EPOD_BASE_DATE = "2024-12-31"

# ✅ Starting Inventory by Season (Updated for SEASON_YR format)
STARTING_INVENTORY_BY_SEASON = {
    "SP24": 80,  # Spring 2024
    "SU24": 60,  # Summer 2024
    "FA24": 40,  # Fall 2024
    "HO24": 50   # Holiday 2024
}

# ✅ Starting Contract Quantities per Channel per Season (Updated for SEASON_YR format)
STARTING_CONTRACT_QTY_BY_SEASON_CHANNEL = {
    "SP24": {"NIKE DIRECT DIGITAL": 50, "NIKE DIRECT STORES": 0, "WHOLESALE": 20},
    "SU24": {"NIKE DIRECT DIGITAL": 40, "NIKE DIRECT STORES": 25, "WHOLESALE": 15},
    "FA24": {"NIKE DIRECT DIGITAL": 30, "NIKE DIRECT STORES": 20, "WHOLESALE": 10},
    "HO24": {"NIKE DIRECT DIGITAL": 45, "NIKE DIRECT STORES": 25, "WHOLESALE": 15},
}

# ✅ Assign SEASON_YR Based on Week Number (SSYY format)
def get_season_yr(week):
    year = "24"  # Assuming 2024
    if 1 <= week <= 13:  # Jan-Mar
        return f"SP{year}"  # Spring
    elif 14 <= week <= 26:  # Apr-Jun
        return f"SU{year}"  # Summer
    elif 27 <= week <= 39:  # Jul-Sep
        return f"FA{year}"  # Fall
    else:  # Oct-Dec
        return f"HO{year}"  # Holiday

# ✅ Function to get the **Saturday date** for each week
def get_weekly_date(week):
    base_date = datetime(2024, 1, 6)  # **First Saturday of 2024**
    return (base_date + timedelta(weeks=week - 1)).strftime("%Y-%m-%d")

# ✅ Generate unique Product_Cds
def generate_product_cds():
    return [f"{''.join(random.choices(string.ascii_uppercase + string.digits, k=6))}-{''.join(random.choices(string.digits, k=3))}" for _ in range(NUM_PRODUCTS)]

# ✅ Generate unique Size_Cds
def generate_size_cds():
    return [str(size) for size in range(8, 8 + NUM_SIZES)]

# ✅ Assign EPOD dynamically (20% to 2024-12-31, 80% later)
def assign_epod_to_products(product_cds):
    num_epod_products = max(1, int(len(product_cds) * 0.2))
    epod_assignment = {}

    epod_products = random.sample(product_cds, num_epod_products)
    for product in product_cds:
        if product in epod_products:
            epod_assignment[product] = EPOD_BASE_DATE
        else:
            epod_assignment[product] = (datetime.strptime(EPOD_BASE_DATE, "%Y-%m-%d") + timedelta(days=365)).strftime("%Y-%m-%d")
    return epod_assignment

# ✅ Generate unique Sold-To-Numbers
def generate_sold_to_numbers():
    sold_to_numbers = {}
    for channel in CHANNELS:
        num_sold_to = NUM_SOLD_TO_NUMBERS // len(CHANNELS)
        sold_to_numbers[channel] = [
            f"000000{np.random.randint(1000, 9999)}" if channel.startswith("NIKE DIRECT") else f"{np.random.randint(1, 9)}{np.random.randint(100000000, 999999999)}"
            for _ in range(num_sold_to)
        ]
    return sold_to_numbers

# ✅ Generate arriving inventory
def generate_arriving_inventory():
    return {week: np.random.choice([0, np.random.randint(20, 50)], p=[0.5, 0.5]) for week in range(1, NUM_WEEKS + 1)}

# CONTRACT SIMULATION

In [118]:
# ✅ Function to determine selling price based on EPOD
def determine_selling_price(channel, product_cd, epod_dict, week):
    epod_date = datetime.strptime(epod_dict[product_cd], "%Y-%m-%d")
    liquidation_start_date = epod_date - timedelta(days=30)  # ✅ New Logic: Liquidation starts 30 days before EPOD

    current_date = datetime.strptime(get_weekly_date(week), "%Y-%m-%d")  # ✅ Convert Date to datetime

    # ✅ Apply liquidation price if the current date is within 30 days of EPOD
    return LIQUIDATION_SELLING_PRICE if current_date >= liquidation_start_date else SELLING_PRICE_BY_CHANNEL[channel]

# ✅ Function to process demand fulfillment
def process_demand(week, channel, product_cd, size_cd, sold_to, arriving_inventory, contract_inventory_dict):
    demand = np.random.randint(5, 20)

    season_yr = get_season_yr(week)  # ✅ Use SSYY format for Season Year
    key = (channel, product_cd, size_cd, season_yr)

    if key not in contract_inventory_dict:
        contract_inventory_dict[key] = STARTING_CONTRACT_QTY_BY_SEASON_CHANNEL[season_yr][channel]

    starting_contract_qty = contract_inventory_dict[key]

    # ✅ Process contract call-offs first
    contract_call_off_qty = min(demand, starting_contract_qty)
    remaining_demand = demand - contract_call_off_qty
    ending_contract_qty = starting_contract_qty - contract_call_off_qty

    # ✅ Process remaining demand from shared inventory
    captured_demand = min(remaining_demand, arriving_inventory)
    lost_demand = remaining_demand - captured_demand
    arriving_inventory -= captured_demand

    # ✅ Ensure contract inventory is rolled over correctly
    contract_inventory_dict[key] = ending_contract_qty

    return {
        "Starting Contract Qty": starting_contract_qty,  # ✅ Correct field placement
        "Contract Call-Off Qty": contract_call_off_qty,
        "Ending Contract Qty": ending_contract_qty,
        "Captured Demand": contract_call_off_qty + captured_demand,
        "Lost Demand": lost_demand,
        "Remaining Inventory": arriving_inventory,
    }

# ✅ Master Simulation Function
def run_simulation(product_cds, size_cds, sold_to_numbers, arriving_inventory_dict, epod_dict):
    cumulative_holding_cost = 0
    holding_cost_per_unit = 2
    cumulative_captured_demand = 0
    cumulative_lost_demand = 0
    contract_inventory_dict = {}  # ✅ Track Contract Inventory Per (Channel, Product, Size, Season)

    data = []

    for week in range(1, NUM_WEEKS + 1):
        season_yr = get_season_yr(week)  # ✅ FIXED: Use SEASON_YR
        arriving_inventory = arriving_inventory_dict[week]
        is_first_row = True

        for product_cd in product_cds:
            for size_cd in size_cds:
                for channel, sold_to_list in sold_to_numbers.items():
                    for sold_to in sold_to_list:

                        # ✅ Process Demand
                        demand_data = process_demand(week, channel, product_cd, size_cd, sold_to, arriving_inventory, contract_inventory_dict)

                        # ✅ Determine Selling Price
                        selling_price = determine_selling_price(channel, product_cd, epod_dict, week)

                        # ✅ Compute Holding Cost
                        weekly_holding_cost = (demand_data["Remaining Inventory"] + demand_data["Ending Contract Qty"]) * holding_cost_per_unit
                        cumulative_holding_cost += weekly_holding_cost

                        # ✅ Update cumulative demand tracking
                        cumulative_captured_demand += demand_data["Captured Demand"]
                        cumulative_lost_demand += demand_data["Lost Demand"]

                        # ✅ Append row data with correct field placement
                        data.append({
                            "Date": get_weekly_date(week),  # ✅ NEW: Add Date field (Saturday of the week)
                            "Week": week,
                            "SEASON_YR": season_yr,  # ✅ FIXED TO SEASON_YR
                            "Sold-To-Number": sold_to,
                            "Channel": channel.upper(),
                            "Product_Cd": product_cd,
                            "Size_Cd": size_cd,
                            "Contract Owner": channel if demand_data["Ending Contract Qty"] > 0 else None,
                            "Starting Contract Qty": demand_data["Starting Contract Qty"],  # ✅ Correct Placement
                            "Contract Call-Off Qty": demand_data["Contract Call-Off Qty"],
                            "Ending Contract Qty": demand_data["Ending Contract Qty"],
                            "Demand": demand_data["Captured Demand"] + demand_data["Lost Demand"],
                            "Captured Demand": demand_data["Captured Demand"],
                            "Lost Demand": demand_data["Lost Demand"],
                            "Arriving Inventory": arriving_inventory if is_first_row else 0,
                            "Remaining Inventory": demand_data["Remaining Inventory"],
                            "Selling Price": selling_price,
                            "EPOD": epod_dict[product_cd],
                            "Cumulative Captured Demand": cumulative_captured_demand,
                            "Cumulative Lost Demand": cumulative_lost_demand,
                            "Weekly Holding Cost": weekly_holding_cost,
                            "Cumulative Holding Cost": cumulative_holding_cost
                        })

                        is_first_row = False  # ✅ Ensure arriving inventory is only displayed on the first row per week

    return pd.DataFrame(data)

# EXECUTION

In [119]:
product_cds = generate_product_cds()
size_cds = generate_size_cds()
sold_to_numbers = generate_sold_to_numbers()
arriving_inventory_dict = generate_arriving_inventory()
epod_dict = assign_epod_to_products(product_cds)

df = run_simulation(product_cds, size_cds, sold_to_numbers, arriving_inventory_dict, epod_dict)

from IPython.display import display
display(df)

# ✅ Install xlsxwriter if not installed
try:
    import xlsxwriter
except ModuleNotFoundError:
    !pip install xlsxwriter
    import xlsxwriter

# ✅ Export to Excel
output_filename = "Simulation.xlsx"

# Create an Excel writer object
with pd.ExcelWriter(output_filename, engine="xlsxwriter") as writer:
    df.to_excel(writer, sheet_name="current", index=False)  # Save simulation results in 'current' tab
    pd.DataFrame().to_excel(writer, sheet_name="proposal", index=False)  # Blank 'proposal' tab

print(f"✅ Simulation results saved to: {output_filename}")

# ✅ Print Final Cumulative Metrics
final_cumulative_captured_demand = df["Cumulative Captured Demand"].iloc[-1]
final_cumulative_lost_demand = df["Cumulative Lost Demand"].iloc[-1]
final_cumulative_holding_cost = df["Cumulative Holding Cost"].iloc[-1]

print("\n📊 **Final Simulation Metrics**")
print(f"🔹 Final Cumulative Captured Demand: {final_cumulative_captured_demand}")
print(f"🔸 Final Cumulative Lost Demand: {final_cumulative_lost_demand}")
print(f"💰 Final Cumulative Holding Cost: ${final_cumulative_holding_cost}")

,Date,Week,SEASON_YR,Sold-To-Number,Channel,Product_Cd,Size_Cd,Contract Owner,Starting Contract Qty,Contract Call-Off Qty,...,Captured Demand,Lost Demand,Arriving Inventory,Remaining Inventory,Selling Price,EPOD,Cumulative Captured Demand,Cumulative Lost Demand,Weekly Holding Cost,Cumulative Holding Cost
0,2024-01-06,1,SP24,0000002680,NIKE DIRECT DIGITAL,4E47IB-838,8,NIKE DIRECT DIGITAL,50,10,...,10,0,0,0,50,2025-12-31,10,0,80,80
1,2024-01-06,1,SP24,0000009433,NIKE DIRECT DIGITAL,4E47IB-838,8,NIKE DIRECT DIGITAL,40,5,...,5,0,0,0,50,2025-12-31,15,0,70,150
2,2024-01-06,1,SP24,0000006682,NIKE DIRECT STORES,4E47IB-838,8,None,0,0,...,0,8,0,0,45,2025-12-31,15,8,0,150
3,2024-01-06,1,SP24,0000009084,NIKE DIRECT STORES,4E47IB-838,8,None,0,0,...,0,15,0,0,45,2025-12-31,15,23,0,150
4,2024-01-06,1,SP24,8871375173,WHOLESALE,4E47IB-838,8,WHOLESALE,20,16,...,16,0,0,0,40,2025-12-31,31,23,8,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9355,2024-12-28,52,HO24,0000009433,NIKE DIRECT DIGITAL,IDDNGT-353,10,None,0,0,...,0,8,0,0,50,2025-12-31,62030,50309,0,223254
9356,2024-12-28,52,HO24,0000006682,NIKE DIRECT STORES,IDDNGT-353,10,None,0,0,...,0,19,0,0,45,2025-12-31,62030,50328,0,223254
9357,2024-12-28,52,HO24,0000009084,NIKE DIRECT STORES,IDDNGT-353,10,None,0,0,...,0,5,0,0,45,2025-12-31,62030,50333,0,223254
9358,2024-12-28,52,HO24,8871375173,WHOLESALE,IDDNGT-353,10,None,0,0,...,0,6,0,0,40,2025-12-31,62030,50339,0,223254


✅ Simulation results saved to: Simulation.xlsx

📊 **Final Simulation Metrics**
🔹 Final Cumulative Captured Demand: 62030
🔸 Final Cumulative Lost Demand: 50351
💰 Final Cumulative Holding Cost: $223254


# OPTIMIZATION SIMULATION

## Data Creation

In [105]:
import numpy as np
import pandas as pd
import random
import string
from datetime import datetime, timedelta
from scipy.optimize import linprog

# Set random seed for reproducibility
np.random.seed(42)

# ✅ Global Parameters
NUM_WEEKS = 52
NUM_CHANNELS = 3
NUM_SOLD_TO_NUMBERS = 6
NUM_PRODUCTS = 10
NUM_SIZES = 3

# ✅ Channels - Configurable
CHANNELS = ["NIKE DIRECT DIGITAL", "NIKE DIRECT STORES", "WHOLESALE"][:NUM_CHANNELS]

# ✅ Channel Selling Prices
SELLING_PRICE_BY_CHANNEL = {
    "NIKE DIRECT DIGITAL": 50,
    "NIKE DIRECT STORES": 45,
    "WHOLESALE": 40
}

# ✅ Liquidation Selling Price
LIQUIDATION_SELLING_PRICE = 20

# ✅ End Product Offer Date (EPOD)
EPOD_BASE_DATE = "2024-12-31"

# ✅ Starting Inventory by Season (Updated for Optimization)
STARTING_INVENTORY_BY_SEASON = {
    season: STARTING_INVENTORY_BY_SEASON[season] + sum(STARTING_CONTRACT_QTY_BY_SEASON_CHANNEL[season].values())
    for season in ["SP24", "SU24", "FA24", "HO24"]
}

# ✅ Starting Contract Quantities (Now All Set to 0)
STARTING_CONTRACT_QTY_BY_SEASON_CHANNEL = {
    season: {channel: 0 for channel in CHANNELS} for season in ["SP24", "SU24", "FA24", "HO24"]
}

# ✅ Assign SEASON_YR Based on Week Number
def get_season_yr(week):
    year = "24"
    if 1 <= week <= 13:
        return f"SP{year}"
    elif 14 <= week <= 26:
        return f"SU{year}"
    elif 27 <= week <= 39:
        return f"FA{year}"
    else:
        return f"HO{year}"

# ✅ Generate Unique Product_Cds
def generate_product_cds():
    return [f"{''.join(random.choices(string.ascii_uppercase + string.digits, k=6))}-{''.join(random.choices(string.digits, k=3))}" for _ in range(NUM_PRODUCTS)]

# ✅ Generate Unique Size_Cds
def generate_size_cds():
    return [str(size) for size in range(8, 8 + NUM_SIZES)]

# ✅ Assign EPOD
def assign_epod_to_products(product_cds):
    num_epod_products = max(1, int(len(product_cds) * 0.2))
    epod_assignment = {}

    epod_products = random.sample(product_cds, num_epod_products)
    for product in product_cds:
        epod_assignment[product] = EPOD_BASE_DATE if product in epod_products else \
            (datetime.strptime(EPOD_BASE_DATE, "%Y-%m-%d") + timedelta(days=365)).strftime("%Y-%m-%d")
    return epod_assignment

# ✅ Generate Unique Sold-To-Numbers
def generate_sold_to_numbers():
    sold_to_numbers = {}
    for channel in CHANNELS:
        num_sold_to = NUM_SOLD_TO_NUMBERS // len(CHANNELS)
        sold_to_numbers[channel] = [
            f"000000{np.random.randint(1000, 9999)}" if channel.startswith("NIKE DIRECT") else f"{np.random.randint(1, 9)}{np.random.randint(100000000, 999999999)}"
            for _ in range(num_sold_to)
        ]
    return sold_to_numbers

# ✅ Generate Arriving Inventory
def generate_arriving_inventory():
    return {week: np.random.choice([0, np.random.randint(20, 50)], p=[0.5, 0.5]) for week in range(1, NUM_WEEKS + 1)}

## Simulation

In [106]:
from scipy.optimize import linprog

# ✅ Simplified Optimized Simulation Function
def run_optimized_simulation(product_cds, size_cds, sold_to_numbers, arriving_inventory_dict, epod_dict):
    cumulative_holding_cost = 0
    holding_cost_per_unit = 2
    cumulative_captured_demand = 0
    cumulative_lost_demand = 0

    # ✅ Track inventory separately for each Product_Cd and Size_Cd
    remaining_inventory_dict = {}

    data = []

    for week in range(1, NUM_WEEKS + 1):
        season_yr = get_season_yr(week)

        for product_cd in product_cds:
            for size_cd in size_cds:
                # ✅ Extract arriving inventory & carry over previous week's remaining inventory
                arriving_inventory = arriving_inventory_dict[week]
                key = (product_cd, size_cd)
                previous_remaining_inventory = remaining_inventory_dict.get(key, STARTING_INVENTORY_BY_SEASON[season_yr])
                starting_inventory = previous_remaining_inventory
                total_inventory = starting_inventory + arriving_inventory  # ✅ Total Available Inventory

                # ✅ Calculate demand per channel
                channel_demands = {channel: np.random.randint(5, 20) * len(sold_to_numbers[channel]) for channel in CHANNELS}
                total_demand = sum(channel_demands.values())

                # ✅ Determine Selling Prices
                selling_prices = {channel: determine_selling_price(channel, product_cd, epod_dict, week) for channel in CHANNELS}

                # ✅ Construct Optimization Problem
                demand_vector = np.array([channel_demands[channel] for channel in CHANNELS])
                price_vector = np.array([selling_prices[channel] for channel in CHANNELS])

                # ✅ Objective Function: Maximize Revenue
                c = -price_vector  # Negative for maximization

                # ✅ Constraint: Total Allocated Inventory ≤ Available Inventory
                A_eq = [np.ones(len(CHANNELS))]
                b_eq = [min(total_demand, total_inventory)]

                # ✅ Solve Optimization Model with Integer Constraints
                result = linprog(
                    c, A_eq=A_eq, b_eq=b_eq,
                    bounds=[(0, demand_vector[i]) for i in range(len(CHANNELS))],
                    method='highs'
                )

                if result.success:
                    allocated_qty = np.round(result.x).astype(int)  # ✅ Ensure Integer Allocations
                else:
                    allocated_qty = np.zeros(len(CHANNELS), dtype=int)  # If failure, allocate nothing

                # ✅ Fix Allocation: Ensure it doesn't exceed available inventory
                allocated_sum = sum(allocated_qty)
                if allocated_sum > total_inventory:
                    allocated_qty = np.floor((allocated_qty / allocated_sum) * total_inventory).astype(int)

                # ✅ Ensure Inventory is Used Properly
                remaining_inventory = max(0, total_inventory - sum(allocated_qty))

                # ✅ Store remaining inventory for the next week
                remaining_inventory_dict[key] = remaining_inventory

                # ✅ Compute Holding Cost
                weekly_holding_cost = remaining_inventory * holding_cost_per_unit
                cumulative_holding_cost += weekly_holding_cost

                # ✅ Compute Captured & Lost Demand
                captured_demand = sum(allocated_qty)
                lost_demand = total_demand - captured_demand
                cumulative_captured_demand += captured_demand
                cumulative_lost_demand += lost_demand

                # ✅ Append results for each channel
                is_first_row = True
                for i, channel in enumerate(CHANNELS):
                    data.append({
                        "Date": get_weekly_date(week),
                        "Week": week,
                        "SEASON_YR": season_yr,
                        "Sold-To-Number": None,
                        "Channel": channel.upper(),
                        "Product_Cd": product_cd,
                        "Size_Cd": size_cd,
                        "Demand": demand_vector[i],
                        "Captured Demand": allocated_qty[i],
                        "Lost Demand": demand_vector[i] - allocated_qty[i],
                        "Starting Inventory": starting_inventory if is_first_row else 0,
                        "Arriving Inventory": arriving_inventory if is_first_row else 0,
                        "Remaining Inventory": remaining_inventory if is_first_row else 0,
                        "Selling Price": selling_prices[channel],
                        "EPOD": epod_dict[product_cd],
                        "Cumulative Captured Demand": cumulative_captured_demand,
                        "Cumulative Lost Demand": cumulative_lost_demand,
                        "Weekly Holding Cost": weekly_holding_cost,
                        "Cumulative Holding Cost": cumulative_holding_cost
                    })

                    is_first_row = False  # ✅ Ensure arriving inventory is only displayed once per week

    return pd.DataFrame(data)

## Execution

In [107]:
# ✅ Generate Data
product_cds = generate_product_cds()
size_cds = generate_size_cds()
sold_to_numbers = generate_sold_to_numbers()
arriving_inventory_dict = generate_arriving_inventory()
epod_dict = assign_epod_to_products(product_cds)

# ✅ Run Optimized Simulation
df_optimized = run_optimized_simulation(product_cds, size_cds, sold_to_numbers, arriving_inventory_dict, epod_dict)

# ✅ Display Results
from IPython.display import display
display(df_optimized)

# ✅ Install xlsxwriter if not installed
try:
    import xlsxwriter
except ModuleNotFoundError:
    !pip install xlsxwriter
    import xlsxwriter

# ✅ Export to Excel
output_filename = "Optimized_Simulation.xlsx"

# Create an Excel writer object
with pd.ExcelWriter(output_filename, engine="xlsxwriter") as writer:
    df_optimized.to_excel(writer, sheet_name="optimized", index=False)  # Save optimized results in 'optimized' tab
    pd.DataFrame().to_excel(writer, sheet_name="proposal", index=False)  # Blank 'proposal' tab

print(f"✅ Optimized simulation results saved to: {output_filename}")

# ✅ Print Final Cumulative Metrics
final_cumulative_captured_demand = df_optimized["Cumulative Captured Demand"].iloc[-1]
final_cumulative_lost_demand = df_optimized["Cumulative Lost Demand"].iloc[-1]
final_cumulative_holding_cost = df_optimized["Cumulative Holding Cost"].iloc[-1]

print("\n📊 **Final Optimized Simulation Metrics**")
print(f"🔹 Final Cumulative Captured Demand: {final_cumulative_captured_demand}")
print(f"🔸 Final Cumulative Lost Demand: {final_cumulative_lost_demand}")
print(f"💰 Final Cumulative Holding Cost: ${final_cumulative_holding_cost}")

,Date,Week,SEASON_YR,Sold-To-Number,Channel,Product_Cd,Size_Cd,Demand,Captured Demand,Lost Demand,Starting Inventory,Arriving Inventory,Remaining Inventory,Selling Price,EPOD,Cumulative Captured Demand,Cumulative Lost Demand,Weekly Holding Cost,Cumulative Holding Cost
0,2024-01-06,1,SP24,None,NIKE DIRECT DIGITAL,EGILBL-793,8,22,22,0,150,0,68,50,2025-12-31,82,0,136,136
1,2024-01-06,1,SP24,None,NIKE DIRECT STORES,EGILBL-793,8,36,36,0,0,0,0,45,2025-12-31,82,0,136,136
2,2024-01-06,1,SP24,None,WHOLESALE,EGILBL-793,8,24,24,0,0,0,0,40,2025-12-31,82,0,136,136
3,2024-01-06,1,SP24,None,NIKE DIRECT DIGITAL,EGILBL-793,9,18,18,0,150,0,86,50,2025-12-31,146,0,172,308
4,2024-01-06,1,SP24,None,NIKE DIRECT STORES,EGILBL-793,9,14,14,0,0,0,0,45,2025-12-31,146,0,172,308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4675,2024-12-28,52,HO24,None,NIKE DIRECT STORES,3PWMKJ-333,9,28,0,28,0,0,0,45,2025-12-31,30300,81258,0,6844
4676,2024-12-28,52,HO24,None,WHOLESALE,3PWMKJ-333,9,34,0,34,0,0,0,40,2025-12-31,30300,81258,0,6844
4677,2024-12-28,52,HO24,None,NIKE DIRECT DIGITAL,3PWMKJ-333,10,24,0,24,0,0,0,50,2025-12-31,30300,81358,0,6844
4678,2024-12-28,52,HO24,None,NIKE DIRECT STORES,3PWMKJ-333,10,38,0,38,0,0,0,45,2025-12-31,30300,81358,0,6844


✅ Optimized simulation results saved to: Optimized_Simulation.xlsx

📊 **Final Optimized Simulation Metrics**
🔹 Final Cumulative Captured Demand: 30300
🔸 Final Cumulative Lost Demand: 81358
💰 Final Cumulative Holding Cost: $6844
